In [7]:
import os
import cv2
import numpy as np
from cvl.dataset import OnlineTrackingBenchmark
from cvl.trackers import MOSSETracker, DCFMOSSETracker
from cvl.lib import get_roi, resume_performance, get_arguments
import matplotlib.pyplot as plt
from copy import copy, deepcopy
import torch
from tqdm.notebook import tqdm as tqdm
import json

In [8]:
ds_path = "Mini-OTB/"

In [9]:
seq_list = os.listdir(ds_path)

In [10]:
dataset = OnlineTrackingBenchmark(ds_path)

# 1) Choose the tracker

In [11]:
TRACKER = "mosse" # ["mosse", "resnet", "mobilenet", "alexnet", "vgg16", "hand_f"]

In [12]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [13]:
if TRACKER == "mosse":
    tracker = MOSSETracker()
    squared = False
elif TRACKER in ["resnet", "mobilenet", "alexnet", "vgg16", "hand_f"]:
    tracker = DCFMOSSETracker(dev = device, features = TRACKER)
    squared = False

In [14]:
results = {}

In [15]:
for idx, seq in enumerate(dataset):
    print(f"----------------------------Evaluating video named {seq.sequence_name}----------------------------------")
    bboxes = []
    for frame_idx, frame in enumerate(seq):
        image_color = frame['image']
        if TRACKER == "mosse":
            image = np.sum(image_color, 2) / 3 # grayscale
        else:
            image = np.transpose(np.float64(image_color), (2, 0, 1))
        if frame_idx == 0:
            bbox = copy(frame['bounding_box'])
            aaaa = bbox
            if bbox.width % 2 == 0:
                bbox.width += 1
            if bbox.height % 2 == 0:
                bbox.height += 1
            roi = get_roi(bbox, squared = squared) # get roi slightly bigger that bbox
            tracker.start(image, bbox, roi) # first frame approach
        else:
            tracker.detect(image)
            tracker.update()
        bboxes.append(copy(tracker.bbox))
    ious = dataset.calculate_per_frame_iou(idx, bboxes)
    print("mIou:", np.mean(ious))
    results[idx] = np.mean(ious)

----------------------------Evaluating video named Basketball----------------------------------
mIou: 0.2567502280605729
----------------------------Evaluating video named Biker----------------------------------
mIou: 0.234573727863974
----------------------------Evaluating video named BlurBody----------------------------------
mIou: 0.15907903841074642
----------------------------Evaluating video named BlurCar3----------------------------------
mIou: 0.3712192746692144
----------------------------Evaluating video named Bolt----------------------------------
mIou: 0.49391794604909356
----------------------------Evaluating video named Box----------------------------------
mIou: 0.27778141831294095
----------------------------Evaluating video named CarScale----------------------------------
mIou: 0.29732700039251897
----------------------------Evaluating video named Coke----------------------------------
mIou: 0.1755226718407867
----------------------------Evaluating video named Coupon--

In [16]:
result_file = f"../results/mIoU_results_mosse.json"

In [17]:
with open(result_file, 'w') as fp:
    json.dump(results, fp)

# 2) Load results

In [18]:
with open(result_file) as fp:
    data = json.load(fp)

In [19]:
mIoU = 0

In [20]:
for k,v in data.items():
    mIoU += v

In [21]:
k 

'29'

In [22]:
mIoU / 30

0.3119091316063585